In [2]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import matplotlib.pyplot as plt

#load data
import data_import as data
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#Model for training
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import ( 
    Conv2D, BatchNormalization, Dropout, MaxPool2D, MaxPooling2D,
    Flatten, Dense, Input, Concatenate, LeakyReLU, Add
)

from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [3]:
train = pd.read_csv(os.getcwd()+'/data/train.csv')
test  = pd.read_csv(os.getcwd()+'/data/test.csv')
submission = pd.read_csv(os.getcwd()+'/data/submission.csv')

In [4]:
from keras.utils.np_utils import to_categorical
x_data = (train[[str(i) for i in range(784)]] / 255.).values.reshape(-1, 28, 28, 1)

#레이블 준비하기
y_data = to_categorical(train['digit'].values)

In [6]:
datagen = ImageDataGenerator(
        rotation_range=10,  
        zoom_range = 0.10,  
        width_shift_range=0.1, 
        height_shift_range=0.1)

In [7]:
x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size = 0.1)
print(x_train.shape)
print(x_val.shape)

(1843, 28, 28, 1)
(205, 28, 28, 1)


In [8]:
model = Sequential() #클래스 객체 생성
model.add(Conv2D(32, kernel_size = 3, input_shape = (28, 28, 1)))  #26 * 26 * 32
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(32, kernel_size = 3)) # 24 * 24 * 32
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(32, kernel_size = 5, strides=2, padding='same', activation='relu')) #12 * 12 * 32
model.add(Dropout(0.4)) 

model.add(Conv2D(64, kernel_size = 3))# 10 * 10 * 64
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(64, kernel_size = 3)) # 8 * 8 * 64
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(64, kernel_size = 5, strides=2, padding='same', activation='relu')) # 4 * 4 * 64
model.add(Dropout(0.4)) 

model.add(Conv2D(128, kernel_size = 4)) # 1 * 1 * 128
model.add(LeakyReLU(alpha=0.1))
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(10, activation='softmax'))


model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 32)        25632     
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 32)        0         
____

In [9]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [10]:
from tensorflow.keras.callbacks import LearningRateScheduler
# DECREASE LEARNING RATE EACH EPOCH
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x)

In [11]:
epochs = 45
#fit 메서드로 트레이닝 시작
hist = model.fit_generator(
    datagen.flow(x_train, y_train, batch_size=64),
    epochs=epochs, 
    steps_per_epoch=x_train.shape[0]//64,
    validation_data=(x_val, y_val), 
    callbacks=[annealer], 
    verbose=0
)

print(
    f"CNN: Epochs={epochs:d}, " +
    f"Train accuracy={max(hist.history['acc']):.5f}, " +                                   
    f"Validation accuracy={max(hist.history['val_acc']):.5f}"
)

CNN: Epochs=45, Train accuracy=0.69477, Validation accuracy=0.76585
